# Intermine-Python: Tutorial 2: Adding Constraints to a Query

This is the second tutorial in the Intermine-Python tutorial series. In the previous tutorial we learnt how to select the output columns as per our choice. Now, we will look at adding constraints to our queries to filter the results.

The first query that we will be looking at is to extract a list of all the Drosophilids in the database. We start with creating a service and query object as we had seen earlier. If we want to view all the possible output columns at once we can pass "\*" as a parameter to the select function. This is Intermine's equivalent of SQL's SELECT *.

In [1]:
from intermine.webservice import Service

In [2]:
service = Service("www.flymine.org/flymine/service")
query=service.new_query("Organism") 

In [3]:
query.select('*')

To add a constraint to our query, we can use the add_constraint method available in the Query class. 

In [4]:
query.add_constraint("genus","=","Drosophila")

<BinaryConstraint: Organism.genus = Drosophila>

Now, we will print the results of our query. We will use the technique that we learnt in the first tutorial. You can also reduce the number of output columns by selecting only certain columns as we had seen in the first tutorial.

In [5]:
for row in query.rows():
    print(row)

Organism: commonName='fruit fly' genus='Drosophila' id=1000001 name='Drosophila melanogaster' shortName='D. melanogaster' species='melanogaster' taxonId=7227
Organism: commonName=None genus='Drosophila' id=6000000 name='Drosophila pseudoobscura' shortName='D. pseudoobscura' species='pseudoobscura' taxonId=7237
Organism: commonName=None genus='Drosophila' id=7000000 name='Drosophila simulans' shortName='D. simulans' species='simulans' taxonId=7240
Organism: commonName=None genus='Drosophila' id=7000006 name='Drosophila erecta' shortName='D. erecta' species='erecta' taxonId=7220
Organism: commonName=None genus='Drosophila' id=7000028 name='Drosophila virilis' shortName='D. virilis' species='virilis' taxonId=7244
Organism: commonName=None genus='Drosophila' id=7000054 name='Drosophila yakuba' shortName='D. yakuba' species='yakuba' taxonId=7245
Organism: commonName=None genus='Drosophila' id=7000083 name='Drosophila willistoni' shortName='D. willistoni' species='willistoni' taxonId=7260
Or

We are now going to change the query a little bit. Let's say that we want to extract all the publication information about Droshophilids that have been published since 2010.

In [6]:
query2=service.new_query("Gene")

Since we want to extract all the information related to each publication pass "publications.\*" as our parameter.

In [7]:
query2.select("publications.*")

This is followed by adding the contraints. First we want to limit our search to only Drosophilds and secondly we want to extract only those publications that were published in or after the year 2010.

In [8]:
query2.add_constraint("organism.genus","=","Drosophila")
query2.add_constraint("publications.year",">=","2010")

<BinaryConstraint: Gene.publications.year >= 2010>

In [9]:
for row in query2.rows(size=10):
    print(row)

Gene: publications.abstractText='"Bang-sensitive"mutants of Drosophila display characteristic repertoires of distinct seizure-and-paralysis behaviors upon mechanical shock (Ganetzky &Wu, 1982, Genetics, 100, 597-614). The authors found that each of the bang-sensitive mutants described in this paper (bas, bss, eas, and tko) also displayed similar behavioral repertoires upon exposure to either high or low temperature. These repertoires are composed of interspersed periods of seizure and paralysis, and appear to have interesting parallels with vertebrate epileptiform behavior. Analysis of gynandromorph mosaics of these bang-sensitive mutant flies indicated that anatomical foci required for these two types of behaviors do not totally overlap, as they were separable among mosaic flies. Observations on mosaic and decapitated flies demonstrated an all-or-none expression of the seizure-and-paralysis behaviors, indicating global activity and long-range interactions in the nervous system. Theref

The add_constraint method is an Intermine-y way of querying the database. The add_constraint method comprises of three parts: 
<br/>
path: The path refers to to the attribute that we want to constrain
<br/>
op or operator: Defines how to constrain the path
<br/>
value: The value to be applied on the operator
<br/>
<br/>
Some common operators are =, >, <, ONE OF, NONE OF, CONTAINS etc.

Now, we will take a look at another query. 

In [10]:
query3=service.new_query("Gene")

In [11]:
query3.add_constraint("organism.genus","=","Drosophila")
query3.add_constraint("organism.species","ONE OF",['melanogaster', 'mojavensis'])

<MultiConstraint: Gene.organism.species ONE OF ['melanogaster', 'mojavensis']>

In [12]:
for row in query3.rows(size=10):
    print(row)

Gene: briefDescription=None cytoLocation='-' description=None id=1000415 length=12653 name='zydeco' primaryIdentifier='FBgn0265767' score=None scoreType=None secondaryIdentifier='CG2893' symbol='zyd'
Gene: briefDescription=None cytoLocation='-' description=None id=1004698 length=1951 name=None primaryIdentifier='FBgn0039942' score=None scoreType=None secondaryIdentifier='CG17163' symbol='CG17163'
Gene: briefDescription=None cytoLocation='-' description=None id=1005938 length=12892 name='Rho GTPase activating protein at 1A' primaryIdentifier='FBgn0025836' score=None scoreType=None secondaryIdentifier='CG40494' symbol='RhoGAP1A'
Gene: briefDescription=None cytoLocation='-' description=None id=1007519 length=21475 name='verthandi' primaryIdentifier='FBgn0260987' score=None scoreType=None secondaryIdentifier='CG17436' symbol='vtd'
Gene: briefDescription=None cytoLocation='-' description=None id=1015398 length=14286 name='Maf1' primaryIdentifier='FBgn0267861' score=None scoreType=None secon

When querying the database using consecutive add_constraint methods, by default the constraints are "and"ed together, i.e. a particular instance will be part of the final result only if it satisfies *all* the constraints. However, we have the option to "or" the constraints as well. We can use the set_logic method to do the same. 

We will now look at an example for the same. 

In [13]:
query4=service.new_query("Gene")

In [14]:
query4.select("publications.*")

In [15]:
query4.add_constraint("organism.genus","=","Drosophila") #A
query4.add_constraint("organism.species","=","yakuba") #B
query4.add_constraint("organism.species","=","melanogaster") #C
query4.add_constraint("publications.year",">=","2010") #D

<BinaryConstraint: Gene.publications.year >= 2010>

In [16]:
query4.set_logic("A & (B | C) & D") #you can also use ("A and (B or C) and D)

In [17]:
for row in query4.rows(size=10):
    print(row)

Gene: publications.abstractText='"Bang-sensitive"mutants of Drosophila display characteristic repertoires of distinct seizure-and-paralysis behaviors upon mechanical shock (Ganetzky &Wu, 1982, Genetics, 100, 597-614). The authors found that each of the bang-sensitive mutants described in this paper (bas, bss, eas, and tko) also displayed similar behavioral repertoires upon exposure to either high or low temperature. These repertoires are composed of interspersed periods of seizure and paralysis, and appear to have interesting parallels with vertebrate epileptiform behavior. Analysis of gynandromorph mosaics of these bang-sensitive mutant flies indicated that anatomical foci required for these two types of behaviors do not totally overlap, as they were separable among mosaic flies. Observations on mosaic and decapitated flies demonstrated an all-or-none expression of the seizure-and-paralysis behaviors, indicating global activity and long-range interactions in the nervous system. Theref